# 变量定义

In [1]:
model_name = 'yolov8n'
model_stru = '-p2'
model_cfg = '_train_val'
exp_name = f'{model_name}{model_stru}{model_cfg}'

best_model_path = f'runs/detect/{exp_name}/train/weights/best.pt'
train_log_path = f'runs/detect/{exp_name}/train/results.csv'
video_path = "datasets/video/landing_1.mp4"

print(f'{best_model_path}')
print(f'{train_log_path}')
print(f'{video_path}')

runs/detect/yolov8n-p2_train_val/train/weights/best.pt
runs/detect/yolov8n-p2_train_val/train/results.csv
datasets/video/landing_1.mp4


# 获取视频信息

In [2]:
import cv2
import os
from ultralytics import YOLO

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

frame_width = int(frame_width/2)
frame_height = int(frame_height/2)
print(f"frame_width: {frame_width}")
print(f"frame_height: {frame_height}")
print(f"frame_rate: {frame_rate}")
print(f"total_frames: {total_frames}")

frame_width: 960
frame_height: 504
frame_rate: 30
total_frames: 241


# 视频检测

In [3]:
import cv2
from tqdm import tqdm

# 加载模型
model = YOLO(best_model_path)

# 打开原始视频
cap = cv2.VideoCapture(video_path)

# 创建VideoWriter对象
out_path = f'results/videos/output-detection.mp4'
out = cv2.VideoWriter(out_path, fourcc, frame_rate,
                      (frame_width, frame_height))

# 初始化进度条
with tqdm(total=total_frames, desc="处理进度", ncols=100) as pbar:
    # 遍历视频
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model.predict(frame, verbose=False)
            frame = results[0].plot()
            frame = cv2.resize(
                frame, (frame_width, frame_height))
            out.write(frame)
            pbar.update(1)
        else:
            break

# 释放资源
cap.release()
out.release()

file_size = os.path.getsize(out_path) / (1024 ** 2)
print(f"文件保存至: {out_path} 文件大小: {file_size:.2f} MB")

处理进度:   0%|                                                             | 0/241 [00:00<?, ?it/s]

处理进度: 100%|███████████████████████████████████████████████████| 241/241 [00:05<00:00, 44.62it/s]

文件保存至: results/videos/output-detection.mp4 文件大小: 5.02 MB


# 视频-追踪

In [4]:
import cv2
from tqdm import tqdm

# 加载模型
model = YOLO(best_model_path)

# 打开原始视频
cap = cv2.VideoCapture(video_path)

# 创建VideoWriter对象
out_path = f'results/videos/output-track.mp4'
out = cv2.VideoWriter(out_path, fourcc, frame_rate,
                      (frame_width, frame_height))

# 初始化进度条
with tqdm(total=total_frames, desc="处理进度", ncols=100) as pbar:
    # 遍历视频
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model.track(frame, persist=True, verbose=False)
            frame = results[0].plot()
            frame = cv2.resize(
                frame, (frame_width, frame_height))
            out.write(frame)
            pbar.update(1)
        else:
            break

# 释放资源
cap.release()
out.release()

file_size = os.path.getsize(out_path) / (1024 ** 2)
print(f"文件保存至: {out_path} 文件大小: {file_size:.2f} MB")

处理进度:   0%|                                                             | 0/241 [00:00<?, ?it/s]

处理进度: 100%|███████████████████████████████████████████████████| 241/241 [00:07<00:00, 30.32it/s]

文件保存至: results/videos/output-track.mp4 文件大小: 5.06 MB


# 视频-追踪-光流

In [5]:
from collections import defaultdict
from tqdm import tqdm
import cv2
import numpy as np

# 加载模型
model = YOLO(best_model_path)

# 打开原始视频
cap = cv2.VideoCapture(video_path)

# 创建VideoWriter对象
out_path = f'results/videos/output-track-path.mp4'
out = cv2.VideoWriter(out_path, fourcc, frame_rate,
                      (frame_width, frame_height))

# 存储追踪历史
track_history = defaultdict(lambda: [])

# 初始化进度条
with tqdm(total=total_frames, desc="处理进度", ncols=100) as pbar:
    # 遍历视频
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model.track(frame, persist=True, verbose=False)
            # 如果检测到并追踪到目标
            if results[0].boxes.xywh is not None and results[0].boxes.id is not None:

                # 获取boxes和追踪track IDs
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()

                # 带有追踪结果的图像
                frame = results[0].plot()

                # 绘制轨迹
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]  # track_id的历史轨迹
                    # 追加新跟踪到的点 x, y center point
                    track.append((float(x), float(y)))
                    if len(track) > 30:  # 为每个目标保留30个历史轨迹
                        track.pop(0)

                    # 画出轨迹线
                    points = np.hstack(track).astype(
                        np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [points], isClosed=False, color=(
                        0, 255, 255), thickness=10)
                frame = cv2.resize(
                    frame, (frame_width, frame_height))
                out.write(frame)

            # 检测到但没有追踪到目标
            elif results[0].boxes.xywh is not None:
                frame = results[0].plot()
                frame = cv2.resize(
                    frame, (frame_width, frame_height))
                out.write(frame)

            # 没有检测到也没有追踪到
            else:
                frame = cv2.resize(
                    frame, (frame_width, frame_height))
                out.write(frame)
            pbar.update(1)
        else:
            break

# 释放资源
cap.release()
out.release()

file_size = os.path.getsize(out_path) / (1024 ** 2)
print(f"文件保存至: {out_path} 文件大小: {file_size:.2f} MB")

处理进度:   0%|                                                             | 0/241 [00:00<?, ?it/s]

处理进度: 100%|███████████████████████████████████████████████████| 241/241 [00:07<00:00, 30.58it/s]

文件保存至: results/videos/output-track-path.mp4 文件大小: 5.07 MB


In [6]:
import torch
torch.cuda.empty_cache()